### Objective

In this notebook, we generate samples for the input paramters for thermal analysis. Here, we condsider two heat sources.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from sklearn.cluster import KMeans
from collections import defaultdict
import utility

In [ ]:
def generate_candidates_efficient(lb, ub, num=10000):
    """
    Efficiently generate candidate samples considering dependencies between features.

    Args:
        lb (array): Lower bounds for each dimension.
        ub (array): Upper bounds for each dimension.
        num (int): Number of candidate samples to generate.

    Returns:
        np.ndarray: Array of generated candidate samples with valid positions.
    """

    # Constants
    c_module = 61.4e-3
    d_module = 106e-3

    # Generate scaled samples using Latin Hypercube Sampling
    sampler = qmc.LatinHypercube(d=len(lb))
    X = sampler.random(n=num) * (ub - lb) + lb
    X[:, -1] = np.round(X[:, -1]).astype(int)  # Assuming rounding is needed for the last column

    # Storage for results
    result_design = []

    # Vectorized calculations for t_min, t_max (avoiding divide by zero)
    t_min = np.ones(num) * 1e-3
    t_max = X[:, 1] / X[:, 5] - 1e-3
    valid_t_mask = t_min < t_max

    # Pre-calculate ranges for position sampling
    Xc_min, Xc_max = c_module / 2, X[:, 1] - c_module / 2
    Yc_min, Yc_max = d_module / 2, X[:, 2] - d_module / 2

    # Generate positions once outside the loop for all samples and filter valid ones later
    position_sampler = qmc.LatinHypercube(d=4)
    all_positions = position_sampler.random(n=50 * num).reshape(num, 50, 4)

    for i, (sample, is_valid_t) in enumerate(zip(X, valid_t_mask)):
        if not is_valid_t:
            continue

        t = np.random.uniform(t_min[i], t_max[i])

        # Scale positions for the current sample
        positions = all_positions[i]
        positions[:, 0] = positions[:, 0] * (Xc_max[i] - Xc_min) + Xc_min
        positions[:, 1] = positions[:, 1] * (Yc_max[i] - Yc_min) + Yc_min
        positions[:, 2] = positions[:, 2] * (Xc_max[i] - Xc_min) + Xc_min
        positions[:, 3] = positions[:, 3] * (Yc_max[i] - Yc_min) + Yc_min

        # Check for non-overlapping positions
        xc1, yc1, xc2, yc2 = positions.T
        non_overlapping = (np.abs(xc1 - xc2) > c_module) | (np.abs(yc1 - yc2) > d_module)

        valid_positions = positions[non_overlapping]
        if len(valid_positions) > 0:
            random_index = np.random.randint(len(valid_positions))
            selected_position = valid_positions[random_index]
            result_design.append(np.concatenate([sample, [t], selected_position]))

    return np.array(result_design)

In [ ]:
%%time
lb = np.array([5e-3, 73.7e-3, 127.2e-3, 10e-3, 20e-3, 10])
ub = np.array([30e-3, 307e-3, 530e-3, 39e-3, 50e-3, 50])
new_candidates = generate_candidates_efficient(lb, ub, num=20000)
print(new_candidates.shape[0])